In [1]:
import numpy as np
import time

from utility.path_tracking import search, \
smooth, transform_points_from_image2real, \
transform2robot_frame, is_near, \
send_path_4_drawing, get_distance, \
transform_pos_angle, pid, \
pioneer_robot_model
from vision.tracker import Tracker, get_marker_object, get_markers, CAM_MAT, CAM_DIST
from robot.evolved_robot import EvolvedRobot
import robot.vrep as vrep

In [2]:
vision_thread = Tracker(mid=5,
                    transform=None,
                    mid_aux=0,
                    video_source=-1,
                    capture=False,
                    show=True,
                    debug=False,
                   )
vision_thread.start()

starting Tracker, video source:  -1


In [3]:
def follow_point():
    try:
        grid = np.full((880, 1190), 255)
        lad = 0.05 # look ahead distance in meters (m)
        wheel_axis = 0.11 # wheel axis distance in meters (m)
        wheel_radius = 0.02 # wheel radius in meters (m)
        indx = 0
        theta = 0.0
        count = 0
        om_sp = 0

        d_controller   = pid(kp=0.5, ki=0, kd=0)
        omega_controller = pid(kp=0.5, ki=0, kd=0)

        robot = EvolvedRobot('thymio-II', -1, None, 1, None)

        robot_m = get_marker_object(7)
        while robot_m.realxy() is None:
            # obtain current position of the robot
            robot_m = get_marker_object(7)

        goal_m = get_marker_object(8)
        while goal_m.realxy() is None:
            # obtain goal marker position
            goal_m = get_marker_object(8)
        
        # import pdb; pdb.set_trace();

        # transform robot position to grid system
        robot_current_position = robot_m.realxy()[:2]

        # transform goal position to grid system
        goal_position = goal_m.realxy()[:2]

        # point to track
        path_to_track = np.array([goal_position])

        while not is_near(robot_current_position, goal_position, dist_thresh = 0.05):
            # get robot marker
            robot_m = get_marker_object(7)
            if robot_m.realxy() is not None:
                # update current position of the robot
                robot_current_position = robot_m.realxy()[:2]

            # calculate robot orientation
            theta = 2*np.pi - robot_m.orientation()
            theta = np.arctan2(np.sin(theta), np.cos(theta))
            
            print('r_t: ', theta)
            # path transformation to vehicle coordinates; relative to the robot
            path_transformed = transform2robot_frame(robot_current_position, path_to_track, theta)
            print('p_t', path_transformed)
            # get distance of each carrot point; relative to the robots
            dist = get_distance(path_transformed, np.array([0,0]))           

            # loop to determine which point will be the carrot/goal point
            for i in range(dist.argmin(), dist.shape[0]):
                if dist[i] < lad and indx <= i:
                    indx = i

            # orientation error relative to the robot
            orient_error = np.arctan2(path_transformed[indx,1], path_transformed[indx,0])
            print('e_t: ', orient_error, '\n')
            # PID controller; desired velocity and rotation
            v_sp = d_controller.control(dist[indx])                     
            om_sp = omega_controller.control(orient_error)
            vr, vl = pioneer_robot_model(v_sp, om_sp, wheel_axis, wheel_radius)
            robot.t_set_motors(vl*20, vr*20)
            count += 1
        else:
            print('GOAAAAAAALL !!')
            print('robot_position: ', robot_current_position)
            print('robot_goal: ', goal_position)
            robot.t_stop()   
    finally:
        time.sleep(0.1)

In [6]:
import ctypes
def send_path_4_drawing(path, sleep_time=0.07, clientID=0):
    """ send path to VREP; the bigger the sleep time the 
        more accurate the points are placed but yo
    """
    for i in path:
        point2send = transform_points_from_image2real(i, 4/1000)
        packedData = vrep.simxPackFloats(point2send.flatten())
        raw_bytes = (ctypes.c_ubyte * len(packedData)
                     ).from_buffer_copy(packedData)
        returnCode = vrep.simxWriteStringStream(
            clientID, "path_coord", raw_bytes, vrep.simx_opmode_oneshot)
        time.sleep(sleep_time)

def follow_path():
    vrep.simxFinish(-1)
    clientID = vrep.simxStart(
        '127.0.0.1',
        19997,
        True,
        True,
        5000,
        5)

    if clientID == -1:
        print('Failed connecting to remote API server')
        print('Program ended')
        return

    if (vrep.simxStartSimulation(clientID, vrep.simx_opmode_blocking) == -1):
        print('Failed to start the simulation\n')
        print('Program ended\n')
        return

    try:
        grid = np.full((880, 1190), 255)
        lad = 0.09  # look ahead distance in meters (m)
        wheel_axis = 0.11  # wheel axis distance in meters (m)
        wheel_radius = 0.02  # wheel radius in meters (m)
        _, look_ahead_sphere = vrep.simxGetObjectHandle(
            clientID, 'look_ahead', vrep.simx_opmode_oneshot_wait)
        indx = 0
        theta = 0.0
        count = 0
        om_sp = 0
        d_controller = pid(kp=0.5, ki=0, kd=0)
        omega_controller = pid(kp=0.5, ki=0, kd=0)

        OP_MODE = vrep.simx_opmode_oneshot_wait
        robot = EvolvedRobot('thymio-II', clientID, None, OP_MODE, None)

        robot_m = get_marker_object(7)
        while robot_m.realxy() is None:
            # obtain current position of the robot
            robot_m = get_marker_object(7)

        goal_m = get_marker_object(8)
        while goal_m.realxy() is None:
            # obtain goal marker postion
            goal_m = get_marker_object(8)

        # transform robot position to grid system
        robot_current_position = (robot_m.realxy()[:2]*1000).astype(int)

        # transform goal position to grid system
        goal_position = (goal_m.realxy()[:2]*1000).astype(int)

        # set position of the robot in simulator
        position, orientation = transform_pos_angle(robot_m.realxy()[:2],
                                                    (2*np.pi - robot_m.orientation()))
        robot.v_set_pos_angle(position, orientation)

        # Search for the path in grid system
        _, path = search(grid,
                         (robot_current_position[1],
                          robot_current_position[0]),
                         (goal_position[1],
                          goal_position[0]),
                         cost=1,
                         D=0.5,
                         fnc='Manhattan')

        # Path smoothing
        newpath = smooth(path,
                         grid,
                         weight_data=0.1,
                         weight_smooth=0.6,
                         number_of_iter=1000)

        # transform GRID points to  real (x, y) coordinates
        path_to_track = transform_points_from_image2real(newpath)

        # Send data to VREP
        send_path_4_drawing(newpath, 0.05, clientID)

        # transform GRID goal to real (x, y) coordinates
        goal_position = goal_m.realxy()[:2]

        while not is_near(robot_current_position, goal_position, dist_thresh=0.05):
            # import pdb; pdb.set_trace();
            # get robot marker
            robot_m = get_marker_object(7)
            if robot_m.realxy() is not None:
                # update current position of the robot
                robot_current_position = robot_m.realxy()[:2]

            # calculate robot orientation
            theta = 2*np.pi - robot_m.orientation()
            theta = np.arctan2(np.sin(theta), np.cos(theta))

            # update position and orientation of the robot in vrep
            position, orientation = transform_pos_angle(
                robot_current_position, theta)
            robot.v_set_pos_angle(position, orientation)

            # path transformation to vehicle coordinates; relative to the robot
            path_transformed = transform2robot_frame(
                robot_current_position, path_to_track, theta)

            # get distance of each carrot point; relative to the robots
            dist = get_distance(path_transformed, np.array([0, 0]))

            # loop to determine which point will be the carrot/goal point
            for i in range(dist.argmin(), dist.shape[0]):
                if dist[i] < lad and indx <= i:
                    indx = i
            # pdb.set_trace();
            # mark the carrot with the sphere
            _ = vrep.simxSetObjectPosition(
                clientID,
                look_ahead_sphere,
                -1,
                (path_to_track[indx, 0]*4,
                 path_to_track[indx, 1]*4,
                 0.005),
                vrep.simx_opmode_oneshot
            )
            # orientation error relative to the robot
            orient_error = np.arctan2(path_transformed[indx, 1], path_transformed[indx, 0])
            print(orient_error)
            # PID controller; desired velocity and rotation
            v_sp = d_controller.control(dist[indx])
            om_sp = omega_controller.control(orient_error)
            vr, vl = pioneer_robot_model(v_sp, om_sp, wheel_axis, wheel_radius)

            robot.t_set_motors(vl*30, vr*30)
            count += 1
        else:
            print('GOAAAAAAALL !!')
            print('robot_position: ', robot_current_position)
            print('robot_goal: ', goal_position)
            robot.t_stop()
    finally:
        time.sleep(0.1)
        vrep.simxStopSimulation(clientID, vrep.simx_opmode_blocking)
        vrep.simxFinish(-1)

In [7]:
follow_path()

Robot: 42  position:  [2.9042837372690955, 2.477665294544198, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, 88.85423716182464]
found goal
794
(208.4209, 195, (394, 201))
39
Robot: 42  position:  [2.9042837372690955, 2.477665294544198, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, 86.68221883166458]
2.4169672660101886
Robot: 42  position:  [2.9042837372690955, 2.477665294544198, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, 86.68221883166458]
2.4169672660101886
Robot: 42  position:  [2.9042837372690955, 2.477665294544198, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, 86.68221883166458]
2.4169672660101886
Robot: 42  position:  [2.9042837372690955, 2.477665294544198, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, 86.68221883166458]
2.4169672660101886
Robot: 42  position:  [2.9042837372690955, 2.477665294544198, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, 86.68221883166458]
2.4169672660101886
Robot: 42  position:  [2.904283737269

SetOrientation object: 42  orientation:  [0, 0, 175.9143832200251]
1.0921600781120473
Robot: 42  position:  [2.8209194508480784, 2.5045994316003006, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, 175.9143832200251]
1.0921600781120473
Robot: 42  position:  [2.8209194508480784, 2.5045994316003006, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, 175.9143832200251]
1.0921600781120473
Robot: 42  position:  [2.8209194508480784, 2.5045994316003006, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, 175.9143832200251]
1.0921600781120473
Robot: 42  position:  [2.8209194508480784, 2.5045994316003006, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, 175.9143832200251]
1.0921600781120473
Robot: 42  position:  [2.81525616365792, 2.4998141637582814, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, -178.21008939175402]
0.9777155605070019
Robot: 42  position:  [2.81525616365792, 2.4998141637582814, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, -178.2100893

Robot: 42  position:  [2.7519609732248975, 2.3590025252285085, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, -105.03781590358284]
-0.451687704679783
Robot: 42  position:  [2.7519609732248975, 2.3590025252285085, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, -105.03781590358284]
-0.451687704679783
Robot: 42  position:  [2.7492539836258882, 2.342777132245024, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, -103.4310288706813]
-0.5054183907159605
Robot: 42  position:  [2.7492539836258882, 2.342777132245024, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, -103.4310288706813]
-0.5054183907159605
Robot: 42  position:  [2.7492539836258882, 2.342777132245024, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, -103.4310288706813]
-0.5054183907159605
Robot: 42  position:  [2.7492539836258882, 2.342777132245024, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, -103.4310288706813]
-0.5054183907159605
Robot: 42  position:  [2.7492539836258882, 2.34277

SetOrientation object: 42  orientation:  [0, 0, -125.70669140060288]
-0.32112354392980075
Robot: 42  position:  [2.6806123245563978, 2.1761995535412106, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, -125.70669140060288]
-0.32112354392980075
Robot: 42  position:  [2.6806123245563978, 2.1761995535412106, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, -125.70669140060288]
-0.32112354392980075
Robot: 42  position:  [2.669689919281854, 2.16303816677465, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, -128.77417093557423]
-0.26697735604662265
Robot: 42  position:  [2.669689919281854, 2.16303816677465, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, -128.77417093557423]
-0.26697735604662265
Robot: 42  position:  [2.669689919281854, 2.16303816677465, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, -128.77417093557423]
-0.26697735604662265
Robot: 42  position:  [2.669689919281854, 2.16303816677465, 0.1388]
SetOrientation object: 42  orientation:  [0, 0

Robot: 42  position:  [2.5350557515669374, 2.0556877881716273, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, -149.0362434679265]
0.14039106453438038
Robot: 42  position:  [2.5350557515669374, 2.0556877881716273, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, -149.0362434679265]
0.14039106453438038
Robot: 42  position:  [2.5350557515669374, 2.0556877881716273, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, -149.0362434679265]
0.14039106453438038
Robot: 42  position:  [2.5350557515669374, 2.0556877881716273, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, -149.0362434679265]
0.14039106453438038
Robot: 42  position:  [2.520524982654155, 2.0461398556638026, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, -149.64188454123342]
0.1638309476620767
Robot: 42  position:  [2.520524982654155, 2.0461398556638026, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, -149.64188454123342]
0.1638309476620767
Robot: 42  position:  [2.520524982654155, 2.0461

...

Robot: 42  position:  [1.9670453550034488, 1.4523175945986804, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, -153.0723221489596]
-0.8787360544304935
Robot: 42  position:  [1.9456706049338022, 1.445162447080089, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, -160.2920211570212]
-0.7882566986095126
Robot: 42  position:  [1.9456706049338022, 1.445162447080089, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, -160.2920211570212]
-0.7882566986095126
Robot: 42  position:  [1.9456706049338022, 1.445162447080089, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, -160.2920211570212]
-0.7882566986095126
Robot: 42  position:  [1.9342934233154856, 1.44594428616817, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, -165.25643716352928]
-0.7137159074544731
Robot: 42  position:  [1.9342934233154856, 1.44594428616817, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, -165.25643716352928]
-0.7137159074544731
Robot: 42  position:  [1.9342934233154856, 1.445944

SetOrientation object: 42  orientation:  [0, 0, 110.22485943116807]
1.3973192719556398
Robot: 42  position:  [1.749924914306873, 1.6825755686401167, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, 110.22485943116807]
1.3973192719556398
Robot: 42  position:  [1.749924914306873, 1.6825755686401167, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, 110.22485943116807]
1.3973192719556398
Robot: 42  position:  [1.741567754173647, 1.6978115467399069, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, 114.32557523912611]
1.373342024150103
Robot: 42  position:  [1.741567754173647, 1.6978115467399069, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, 114.32557523912611]
1.373342024150103
Robot: 42  position:  [1.741567754173647, 1.6978115467399069, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, 114.32557523912611]
1.373342024150103
Robot: 42  position:  [1.7319420491787776, 1.7116149481743483, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, 119.12405349

SetOrientation object: 42  orientation:  [0, 0, -135.00000000000003]
-0.74053832764234
Robot: 42  position:  [1.5274622314194255, 1.6627756998618715, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, -131.9872124958167]
-0.8290272863079695
Robot: 42  position:  [1.5274622314194255, 1.6627756998618715, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, -131.9872124958167]
-0.8290272863079695
Robot: 42  position:  [1.5274622314194255, 1.6627756998618715, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, -131.9872124958167]
-0.8290272863079695
Robot: 42  position:  [1.518320676262354, 1.6495700850863675, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, -130.52510349375393]
-0.8920003427473872
Robot: 42  position:  [1.518320676262354, 1.6495700850863675, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, -130.52510349375393]
-0.8920003427473872
Robot: 42  position:  [1.518320676262354, 1.6495700850863675, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, 

Robot: 42  position:  [1.3284179278402035, 1.553780988608317, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, 177.13759477388825]
-0.06886035738137747
Robot: 42  position:  [1.3284179278402035, 1.553780988608317, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, 177.13759477388825]
-0.06886035738137747
Robot: 42  position:  [1.3284179278402035, 1.553780988608317, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, 177.13759477388825]
-0.06886035738137747
Robot: 42  position:  [1.3140498886219674, 1.5563615091906304, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, 174.28940686250036]
-0.03257514728370707
Robot: 42  position:  [1.3140498886219674, 1.5563615091906304, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, 174.28940686250036]
-0.03257514728370707
Robot: 42  position:  [1.3140498886219674, 1.5563615091906304, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, 174.28940686250036]
-0.03257514728370707
Robot: 42  position:  [1.29923892555988, 1.

SetOrientation object: 42  orientation:  [0, 0, 169.38034472384487]
0.18909914809118217
Robot: 42  position:  [1.0417861918618818, 1.6211320650981083, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, 169.38034472384487]
0.18909914809118217
Robot: 42  position:  [1.0253799427650465, 1.623146981244165, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, 170.0737544933483]
0.18427255805181791
Robot: 42  position:  [1.0253799427650465, 1.623146981244165, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, 170.0737544933483]
0.18427255805181791
Robot: 42  position:  [1.0253799427650465, 1.623146981244165, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, 170.0737544933483]
0.18427255805181791
Robot: 42  position:  [1.0253799427650465, 1.623146981244165, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, 170.0737544933483]
0.18427255805181791
Robot: 42  position:  [1.0089965983199503, 1.6247075268520847, 0.1388]
SetOrientation object: 42  orientation:  [0, 0, 171.46